# 02 WIP: Development

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
os.chdir("..")
os.getcwd()

In [ ]:
from src.gridgpt.template_manager import (
    load_templates,
    select_template,
    print_template_grid,
    identify_theme_slots
)

# 01 Create word database

1. In the terminal, navigate to the root directory of the project: `cd /path/to/gridgpt`
2. Activate your virtual environment:

   ```bash
   source venv/bin/activate  # On macOS/Linux
   .\venv\Scripts\activate  # On Windows
   ```

2. Run the `create_word_db` script to generate the word database. Use the following parameters:
- `--min_frequency` to set the minimum frequency of words to include in the database as indicated in the original crossword word database.
- `--min_length` to set the minimum number of characters a word can have to be included in the database.
- `--max_length` to set the maximum number of characters a word can have to be included in the database.

   ```bash
   python scripts/create_word_db.py --min-frequency 2 --min-length 3 --max-length 5
   ```

The following files will be created as a result from running the script: 

1. [`word_database_all.json`](../data/02_intermediary/word_database/word_database_all.json): A JSON file containing all words and their frequencies from the original crossword word database.
2. [`word_database_filtered_with_frequencies.json`](../data/02_intermediary/word_database/word_database_filtered_with_frequencies.json): A JSON file containing words filtered by length and minimum frequency; all words as keys with their respective frequencies included as values in a single dictionary.
3. [`word_database_filtered_by_length.json`](../data/02_intermediary/word_database/word_database_filtered_by_length.json): A JSON file containing words filtered by length and minimum frequency; keys are the lengths of the words, and values are lists of words of that length.
4. [`word_database_filtered_list.json`](../data/02_intermediary/word_database/word_database_filtered_list.json): A JSON file containing words filtered by length and minimum frequency; stored as a simple list of all words that meet the criteria.

# 01 Get template

In [ ]:
templates_data = load_templates()
templates = templates_data["templates"]

for template in templates:
    print(f"{template['name']} (ID: {template['id']}, difficulty: {template['difficulty']})")

In [ ]:
# template = select_template(difficulty="easy")
template = select_template(template_id='5x5_basic')

In [ ]:
print_template_grid(template)

# Crossword Generator

In [ ]:
from src.gridgpt.crossword_generator import CrosswordGenerator
from src.gridgpt.crossword_generator import generate_themed_crossword

## Initialize generator

In [ ]:
generator = CrosswordGenerator()

## Set theme

In [ ]:
theme_entry = "BREWS"

In [ ]:
generator.validate_theme_entry(theme_entry)

In [ ]:
template_with_theme = generator.place_theme_entry(template, theme_entry)
template_with_theme

In [ ]:
crossword = generate_themed_crossword(template, theme_entry)

In [ ]:
crossword

In [ ]:
#!/usr/bin/env python3
import argparse
import json
import logging
import os
import sys

from src.gridgpt.template_manager import load_templates, select_template, print_template_grid
from src.gridgpt.crossword_generator import generate_themed_crossword

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def parse_args():
    """Parse command-line arguments."""
    parser = argparse.ArgumentParser(description='Generate a crossword puzzle')
    
    parser.add_argument(
        '--template', 
        help='ID of template to use (default: random)'
    )
    
    parser.add_argument(
        '--theme-entry',
        help='Theme entry to include in the crossword'
    )
    
    parser.add_argument(
        '--difficulty',
        choices=['easy', 'medium', 'hard'],
        help='Difficulty level of the crossword'
    )
    
    parser.add_argument(
        '--output',
        help='Output file to save the crossword (JSON format)'
    )
    
    parser.add_argument(
        '--verbose', 
        '-v', 
        action='store_true',
        help='Enable verbose logging'
    )
    
    return parser.parse_args()

def main():
    """Main function to generate a crossword puzzle."""
    # Parse command-line arguments
    args = parse_args()
    
    # Set logging level
    if args.verbose:
        logging.getLogger().setLevel(logging.DEBUG)
    
    # Load templates
    try:
        templates_data = load_templates()
    except Exception as e:
        logger.error(f"Failed to load templates: {e}")
        sys.exit(1)
    
    # Select template
    try:
        template = select_template(templates_data, template_id=args.template, difficulty=args.difficulty)
    except Exception as e:
        logger.error(f"Failed to select template: {e}")
        sys.exit(1)
    
    # Print the selected template
    print_template_grid(template)
    
    # Generate crossword
    try:
        crossword = generate_themed_crossword(template, args.theme_entry)
    except ValueError as e:
        logger.error(f"Failed to generate crossword: {e}")
        sys.exit(1)
    
    # Save to output file if specified
    if args.output:
        try:
            with open(args.output, 'w', encoding='utf-8') as f:
                json.dump(crossword, f, indent=2)
            logger.info(f"Crossword saved to {args.output}")
        except Exception as e:
            logger.error(f"Failed to save crossword: {e}")
    
    return 0

if __name__ == "__main__":
    sys.exit(main())

# Generate Clues

In [ ]:
from src.gridgpt.clue_generator import generate_mixed_clues

In [ ]:
theme = "general knowledge"
clues = x(crossword, theme)

In [ ]:
# Print clues
print("\nClues:")
print("\nAcross:")
across_clues = {k: v for k, v in clues.items() if 'A' in k}
for slot_id, clue in sorted(across_clues.items()):
    word = crossword["filled_slots"][slot_id]
    print(f"{slot_id}: {clue} ({word})")
    
print("\nDown:")
down_clues = {k: v for k, v in clues.items() if 'D' in k}
for slot_id, clue in sorted(down_clues.items()):
    word = crossword["filled_slots"][slot_id]
    print(f"{slot_id}: {clue} ({word})")

In [ ]:
clues

In [ ]:
# order keys by slot ID
ordered_clues = {k: clues[k] for k in sorted(clues.keys())}
ordered_clues

In [ ]:
crossword['grid']

In [ ]:
clues

In [ ]:
# TODO: add nyt sample clues, add clue character limits and add nyt clue types (abbr, language, quotes, etc)
# TODO: theme manager: input theme -> generate theme entry
# TODO: database, add similarity score of theme to words (in addition to frequency) and use that to weigh word choice in addition to frequency